In [1]:
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader, PDDLWriter
from VSLAM import *

from copy import deepcopy
from itertools import combinations
import random
import os

import matplotlib.pyplot as plt

from meta_planning.parsers import parse_trajectory, parse_model
from meta_planning import LearningTask
from meta_planning.evaluation import SynEvaluator

In [2]:
def evaluate_f1score(positives, negatives, FAMA_pre, FAMA_eff):

    tp_FAMA = 0
    fp_FAMA = 0
    tn_FAMA = 0
    fn_FAMA = 0
    
    for demonstration in positives:
        action_name = demonstration[1][0]

        if is_consistent(FAMA_pre[action_name], FAMA_eff[action_name], demonstration):
            tp_FAMA += 1
        else:
            fn_FAMA += 1

    for demonstration in negatives:
        action_name = demonstration[1][0]

        if is_consistent(FAMA_pre[action_name], FAMA_eff[action_name], demonstration):
            tn_FAMA += 1
        else:
            fp_FAMA += 1

    if tp_FAMA + fp_FAMA == 0:
        precision_FAMA = 1
    else:
        precision_FAMA = tp_FAMA / (tp_FAMA + fp_FAMA)
    recall_FAMA = tp_FAMA / (tp_FAMA + fn_FAMA)

    print(precision_FAMA, recall_FAMA)

    f1_FAMA = (2 * precision_FAMA * recall_FAMA) / (precision_FAMA + recall_FAMA)

    print(round(f1_FAMA,2))

    return f1_FAMA

In [3]:
def encode_as_hypothesis(problem, learned_model):
    action_params = {action.name : {"var%s" % str(i+1): action.parameters[i].name for i in range(len(action.parameters))} for action in problem.actions}

    FAMA_pre = dict()
    FAMA_eff = dict()
    for schema in learned_model.schemata:
        action_name = schema.name.replace("-", "_")

        propositions = schema.propositional_preconditions()
        precondition = []
        for proposition in propositions:
            splitted_proposition = proposition.predicate.split("_")
            predicate = splitted_proposition[2].replace("-", "_")
            args = tuple([action_params[action_name][par] for par in splitted_proposition[3:]])

            literal = (predicate, args, True)
            precondition += [literal]
        FAMA_pre[action_name] = frozenset(precondition)

        propositions = schema.propositional_effects()
        effects = []
        for proposition in propositions:
            splitted_proposition = proposition.predicate.split("_")
            predicate = splitted_proposition[2].replace("-", "_")
            args = tuple([action_params[action_name][par] for par in splitted_proposition[3:]])

            literal = (predicate, args, True)

            if literal in FAMA_pre[action_name]:
                effects += [(predicate, args, False)]
            else:
                effects += [literal]
        FAMA_eff[action_name] = frozenset(effects)

    return FAMA_pre, FAMA_eff

### CHOOSE DOMAIN AND RATE HERE

In [4]:
domain_name = "blocks"
get_environment().credits_stream = None
reader = PDDLReader()

# fix_dashes(domain_name)

We use the same demonstrations used for VSLAM to generate state trajectories (FAMA's data structure)

In [6]:
lifted_to_int = dict()

pos_demonstrations = set()
neg_demonstrations = set()
problems = sorted(os.listdir("benchmarks/{}/problems/".format(domain_name)))
for problem_file in problems:
    print(problem_file)
    problem = reader.parse_problem('benchmarks/{}/domain.pddl'.format(domain_name), 'benchmarks/{}/problems/{}'.format(domain_name, problem_file))

    new_transitions, new_failures = generate_transitions_from_problem(problem)
    new_pos, lifted_to_grounded = lift_transitions_with_map(new_transitions, problem.actions)
    new_neg = lift_transitions(new_failures, problem.actions)

    for pos_demonstration in new_pos:
        if lifted_to_int.get(pos_demonstration, -1) == -1:
            next_value = len(lifted_to_int.keys())
            lifted_to_int[pos_demonstration] = next_value
            


            problem_objects = ["{} - {}".format(o.name, o.type) for o in problem.all_objects]
            transition = lifted_to_grounded[pos_demonstration]

            pre_state = ["({} {})".format(literal[0].replace("_", "-"), " ".join(literal[1])) for literal in transition[0] if literal[2]]
            action = "({} {})".format(transition[1][0].replace("_", "-"), " ".join(map(str,transition[1][1])))
            post_state = ["({} {})".format(literal[0].replace("_", "-"), " ".join(literal[1])) for literal in transition[2] if literal[2]]

            trajectory_str = ""
            trajectory_str += "(trajectory\n\n"
            trajectory_str += "(:objects {})\n\n".format(" ".join(problem_objects))
            trajectory_str += "(:init {})\n\n".format(" ".join(pre_state))
            trajectory_str += "(:action {})\n\n".format(action)
            trajectory_str += "(:state {})\n\n".format(" ".join(post_state))
            trajectory_str += ")"

            with open("benchmarks/{}/FAMA/trajectory-{}".format(domain_name, str(next_value).zfill(3) ), "w") as f:
                f.write(trajectory_str)



    pos_demonstrations.update(new_pos)
    neg_demonstrations.update(new_neg)

problem-00.pddl
SequentialPlan:
    unstack(b8, b3, b1)
    putdown(b8, b1, b2)
    unstack(b3, b4, b1)
    putdown(b3, b1, b2)
    unstack(b7, b2, b1)
    putdown(b7, b1, b2)
    unstack(b2, b6, b1)
    putdown(b2, b1, b3)
    unstack(b6, b5, b1)
    putdown(b6, b1, b2)
    unstack(b5, b1, b2)
    stack(b5, b4, b1)
    pickup(b1, b2, b3)
    stack(b1, b3, b2)
    pickup(b6, b1, b2)
    stack(b6, b5, b1)
    pickup(b2, b1, b3)
    stack(b2, b6, b1)
    pickup(b7, b1, b2)
    stack(b7, b2, b1)
From 20 transitions to 13 lifted transitions
From 80 transitions to 71 lifted transitions
problem-01.pddl
SequentialPlan:
    unstack(b5, b3, b1)
    putdown(b5, b1, b2)
    unstack(b7, b6, b1)
    stack(b7, b5, b1)
    unstack(b3, b2, b1)
    putdown(b3, b1, b2)
    unstack(b2, b1, b3)
    putdown(b2, b1, b3)
    pickup(b6, b1, b2)
    stack(b6, b3, b1)
    unstack(b7, b5, b1)
    putdown(b7, b1, b2)
    pickup(b5, b1, b2)
    stack(b5, b2, b1)
    unstack(b4, b8, b1)
    stack(b4, b1, b2)
    pi

Shuffling

In [7]:
pos_demonstrations = sorted(list(pos_demonstrations))
neg_demonstrations = sorted(list(neg_demonstrations))

random.seed(12345)
random.shuffle(pos_demonstrations)
random.shuffle(neg_demonstrations)

print("{} Demonstrations ({} positive, {} negative)".format(len(pos_demonstrations) + len(neg_demonstrations), len(pos_demonstrations), len(neg_demonstrations)))

948 Demonstrations (66 positive, 882 negative)


Preparation of the training and testing set of demonstrations using a 50/50 split

In [8]:
all_actions = problem.actions
all_fluents = problem.fluents
static_fluents = problem.get_static_fluents()

ratio_training_test = 0.5
ratio_neg_pos = 10

pos_cut = int(len(pos_demonstrations)*ratio_training_test)
neg_cut = int(len(neg_demonstrations)*ratio_training_test)


training_pos_demonstrations = pos_demonstrations[:pos_cut]
testing_pos_demonstrations = pos_demonstrations[pos_cut:]
training_neg_demonstrations = neg_demonstrations[:neg_cut]
testing_neg_demonstrations = neg_demonstrations[neg_cut:]


training_neg_demonstrations = training_neg_demonstrations[:len(training_pos_demonstrations)*ratio_neg_pos]


seen_actions = {action.name:False for action in all_actions}

initial_pos_demonstrations = []
noninitial_pos_demonstrations = []

for demonstration in training_pos_demonstrations:
    action_name = demonstration[1][0]
    if not seen_actions[action_name]:
        initial_pos_demonstrations += [demonstration]
        seen_actions[action_name] = True
    else:
        noninitial_pos_demonstrations += [demonstration]


### Build demonstration set by spacing out pos and neg demonstrations
snapshots = []
slice_length = len(training_neg_demonstrations)/len(noninitial_pos_demonstrations)
demonstrations = []
for i in range(len(noninitial_pos_demonstrations)):
    start = int(slice_length * i)
    end = int(slice_length * (i+1))
    
    demonstrations += training_neg_demonstrations[start:end]
    demonstrations += noninitial_pos_demonstrations[i:i+1]
    snapshots += [end+i]

Initialization

In [16]:
M = parse_model("benchmarks/{}/FAMA/domain.pddl".format(domain_name))
T = [parse_trajectory("benchmarks/{}/FAMA/trajectory-{}".format(domain_name, str(lifted_to_int[demonstration]).zfill(3)),M) for demonstration in initial_pos_demonstrations]

Learning and evaluation

In [17]:
FAMA_results = []

for i in snapshots:
    demonstration = demonstrations[i]   
    action_name = demonstration[1][0]

    M = parse_model("benchmarks/{}/FAMA/domain.pddl".format(domain_name))
    T += [parse_trajectory("benchmarks/{}/FAMA/trajectory-{}".format(domain_name, str(lifted_to_int[demonstration]).zfill(3)),M)]
    O = [t.observe(1) for t in T]

    LT = LearningTask(M,O)
    try:
        solution = LT.learn()
        if solution.solution_found:
            learned_model = solution.learned_model
            FAMA_pre, FAMA_eff = encode_as_hypothesis(problem, learned_model)
            FAMA_f1 = evaluate_f1score(testing_pos_demonstrations, testing_neg_demonstrations, FAMA_pre, FAMA_eff)
        else:
            FAMA_f1 = float('NaN')

    except:
        FAMA_f1 = float('NaN')
    
    FAMA_results += [FAMA_f1]

ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 11 -T 13 > planner_out
1.0 0.5151515151515151
0.68
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 13 -T 15 > planner_out
1.0 0.5151515151515151
0.68
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 15 -T 17 > planner_out


1.0 0.5757575757575758
0.73
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 17 -T 19 > planner_out
1.0 0.6060606060606061
0.75
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 19 -T 21 > planner_out
1.0 0.9393939393939394
0.97
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 21 -T 23 > planner_out


1.0 0.8181818181818182
0.9
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 23 -T 25 > planner_out
1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 25 -T 27 > planner_out
1.0 1.0
1.0


ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 27 -T 29 > planner_out
1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 29 -T 31 > planner_out
1.0 1.0
1.0


ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 31 -T 33 > planner_out
1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 33 -T 35 > planner_out


...

1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 59 -T 61 > planner_out
1.0 1.0
1.0


ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 61 -T 63 > planner_out
1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 63 -T 65 > planner_out


1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 65 -T 67 > planner_out


1.0 1.0
1.0
ulimit -t 3000; /home/dieaigar/Work/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 67 -T 69 > planner_out


1.0 1.0
1.0


Store the results

In [ ]:
with open("benchmarks/{}/f1_FAMA_results.csv".format(domain_name), "w") as f:
    for i in range(len(FAMA_results)):
        f.write("{}\n".format(FAMA_results[i]))